### Predicting Kicker Points

In [45]:
#import libraries
import nfl_data_py as nfl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
%matplotlib inline
style.use('fivethirtyeight')
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [46]:
seasons = range(2020,2024)
pbp_df = nfl.import_pbp_data(seasons)

2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [47]:
pbp_df.columns.values

array(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive',
       'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo',
       'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun',
       'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble',
       'pass_length', 'pass_location', 'air_yards', 'yards_after_catch',
       'run_location', 'run_gap', 'field_goal_result', 'kick_distance',
       'extra_point_result', 'two_point_conv_result',
       'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout',
       'timeout_team', 'td_team', 'td_player_name', 'td_player_id',
       'posteam_timeouts_remaining', 'defteam_timeouts_remaining',
       'total_home_score', 'total_away_score', 'posteam_score'

In [48]:
cols = ['game_id','home_team','away_team','yardline_100','game_seconds_remaining','ydstogo','play_type','kick_distance','extra_point_result','two_point_conv_result','extra_point_prob','two_point_conversion_prob','two_point_attempt','field_goal_attempt','kicker_player_name','defensive_two_point_attempt','defensive_extra_point_attempt', 'defensive_extra_point_conv', 'defensive_two_point_conv','roof', 'surface', 'temp', 'wind']
        

In [49]:
pbp_kicker = pbp_df[cols]
pbp_kicker = pbp_kicker.query("play_type == 'extra_point' | play_type == 'field_goal' ").reset_index()

In [50]:
pd.set_option('display.max_columns', None)
pbp_kicker.head(20)

,index,game_id,home_team,away_team,yardline_100,game_seconds_remaining,ydstogo,play_type,kick_distance,extra_point_result,two_point_conv_result,extra_point_prob,two_point_conversion_prob,two_point_attempt,field_goal_attempt,kicker_player_name,defensive_two_point_attempt,defensive_extra_point_attempt,defensive_extra_point_conv,defensive_two_point_conv,roof,surface,temp,wind
0,7,2020_01_ARI_SF,SF,ARI,34.0,3416.0,3.0,field_goal,52.0,None,None,0.000000,0.0,0.0,1.0,R.Gould,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
1,16,2020_01_ARI_SF,SF,ARI,15.0,3293.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,R.Gould,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
2,27,2020_01_ARI_SF,SF,ARI,15.0,3115.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
3,62,2020_01_ARI_SF,SF,ARI,34.0,2253.0,5.0,field_goal,52.0,None,None,0.000000,0.0,0.0,1.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
4,82,2020_01_ARI_SF,SF,ARI,6.0,1833.0,2.0,field_goal,24.0,None,None,0.000000,0.0,0.0,1.0,R.Gould,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
5,91,2020_01_ARI_SF,SF,ARI,38.0,1802.0,5.0,field_goal,56.0,None,None,0.000000,0.0,0.0,1.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
6,106,2020_01_ARI_SF,SF,ARI,31.0,1494.0,10.0,field_goal,49.0,None,None,0.000000,0.0,0.0,1.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
7,145,2020_01_ARI_SF,SF,ARI,15.0,626.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
8,155,2020_01_ARI_SF,SF,ARI,15.0,518.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,R.Gould,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
9,166,2020_01_ARI_SF,SF,ARI,15.0,303.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0


In [51]:
pbp_kicker[pbp_kicker['roof']=='outdoors']

,index,game_id,home_team,away_team,yardline_100,game_seconds_remaining,ydstogo,play_type,kick_distance,extra_point_result,two_point_conv_result,extra_point_prob,two_point_conversion_prob,two_point_attempt,field_goal_attempt,kicker_player_name,defensive_two_point_attempt,defensive_extra_point_attempt,defensive_extra_point_conv,defensive_two_point_conv,roof,surface,temp,wind
0,7,2020_01_ARI_SF,SF,ARI,34.0,3416.0,3.0,field_goal,52.0,None,None,0.000000,0.0,0.0,1.0,R.Gould,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
1,16,2020_01_ARI_SF,SF,ARI,15.0,3293.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,R.Gould,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
2,27,2020_01_ARI_SF,SF,ARI,15.0,3115.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
3,62,2020_01_ARI_SF,SF,ARI,34.0,2253.0,5.0,field_goal,52.0,None,None,0.000000,0.0,0.0,1.0,Z.Gonzalez,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
4,82,2020_01_ARI_SF,SF,ARI,6.0,1833.0,2.0,field_goal,24.0,None,None,0.000000,0.0,0.0,1.0,R.Gould,0.0,0.0,0.0,0.0,outdoors,grass,66.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8517,178051,2023_11_TEN_JAX,JAX,TEN,15.0,1452.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,B.McManus,0.0,0.0,0.0,0.0,outdoors,grass,75.0,3.0
8518,178064,2023_11_TEN_JAX,JAX,TEN,15.0,1102.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,B.McManus,0.0,0.0,0.0,0.0,outdoors,grass,75.0,3.0
8519,178073,2023_11_TEN_JAX,JAX,TEN,15.0,900.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,N.Folk,0.0,0.0,0.0,0.0,outdoors,grass,75.0,3.0
8520,178089,2023_11_TEN_JAX,JAX,TEN,15.0,448.0,0.0,extra_point,33.0,good,None,0.932427,0.0,0.0,0.0,B.McManus,0.0,0.0,0.0,0.0,outdoors,grass,75.0,3.0


In [52]:
#cool graphic of kickers doing the best

In [53]:
pbp_kicker['roof'].value_counts()

roof
outdoors    5505
dome        1638
closed       987
open         178
Name: count, dtype: int64

In [54]:
pbp_kicker['temp'].isnull().value_counts()

temp
True     5484
False    3038
Name: count, dtype: int64

In [55]:
pbp_kicker['wind'].isnull().value_counts()

wind
True     5484
False    3038
Name: count, dtype: int64

In [56]:
pbp_kicker['surface'].value_counts()

surface
grass         4592
fieldturf     1862
matrixturf     620
astroturf      606
sportturf      372
               352
a_turf         118
Name: count, dtype: int64

In [57]:
### Exploratory Data Analysis
#### Exploring how distance, roof, surface, temp, and wind affect kicker outcomes
#### too many null temp and wind columns stopped tracking information?
#### Can we predict a kickers ability to score a field goal first? using distance and roof type

In [58]:
#build dataframe
pbp_kicker_pred = pbp_kicker[['extra_point_result','kick_distance','roof']].copy()
#Do this before or after train test split?
replacements = {'good':1, None:0,'failed':0,'blocked':0}
pbp_kicker_pred.extra_point_result = pbp_kicker_pred.extra_point_result.replace(replacements)

In [70]:
#train_test_split
X = pbp_kicker_pred[['kick_distance', 'roof']]
y = pbp_kicker_pred['extra_point_result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [71]:
X_train.roof.value_counts()

roof
outdoors    4119
dome        1230
closed       749
open         130
Name: count, dtype: int64

In [72]:
#Encoding roof variable
X_train['is_outdoors'] = X_train['roof'].apply(lambda x: 1 if x == 'outdoors' else 0)
X_train['is_dome'] = X_train['roof'].apply(lambda x: 1 if x == 'dome' else 0)
X_train['is_closed'] = X_train['roof'].apply(lambda x: 1 if x == 'closed' else 0)

X_test['is_outdoors'] = X_test['roof'].apply(lambda x: 1 if x == 'outdoors' else 0)
X_test['is_dome'] = X_test['roof'].apply(lambda x: 1 if x == 'dome' else 0)
X_test['is_closed'] = X_test['roof'].apply(lambda x: 1 if x == 'closed' else 0)

In [73]:
### Standard Scalar
features = ['kick_distance']

data_reshaped = X_train['kick_distance'].values.reshape(-1, 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_reshaped)
df_scaled = pd.DataFrame(scaled_data)

In [74]:
df_scaled

,0
0,-1.657278
1,0.289982
2,-0.359105
3,-0.359105
4,-0.359105
...,...
6386,2.237241
6387,1.977607
6388,-1.138009
6389,-0.359105


In [75]:
#X_train  = X_train.drop(['kick_distance', 'roof'], axis=1)
X_train = pd.concat([X_train.reset_index(drop=True),df_scaled],axis=1)

In [76]:
X_train.head()
X_train  = X_train.drop(['kick_distance', 'roof'], axis=1)

In [77]:
X_train.head()

,is_outdoors,is_dome,is_closed,0
0,0,1,0,-1.657278
1,1,0,0,0.289982
2,0,0,1,-0.359105
3,1,0,0,-0.359105
4,1,0,0,-0.359105


In [78]:
#y_train = pd.DataFrame(y_train)

In [79]:
#y_train = y_train.reset_index(drop=True)

In [80]:
#y_train.value_counts()

In [84]:
X_train = X_train.rename(columns={0: 'kick_distance'})
X_train = X_train[['kick_distance'] + [col for col in X_train.columns if col != 'kick_distance']]
X_test  = X_test.drop(['roof'], axis=1)

KeyError: "['roof'] not found in axis"

In [85]:
X_test

,kick_distance,is_outdoors,is_dome,is_closed
7501,33.0,1,0,0
7482,33.0,0,0,0
2572,32.0,0,1,0
1084,33.0,0,0,1
856,22.0,0,1,0
...,...,...,...,...
4834,33.0,1,0,0
2199,33.0,0,0,1
7352,33.0,1,0,0
1784,33.0,1,0,0


In [86]:
#Results
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)
print('Intercept of model',logistic_model.intercept_)
print('Coefficient of model :', logistic_model.coef_)

score_train = logistic_model.score(X_train, y_train)
print('accuracy_score overall when tested on the training set:', score_train)

score_test = logistic_model.score(X_test, y_test)
print('accuracy_score overall when tested on the testing set :', score_test)

Intercept of model [-0.00518573]
Coefficient of model : [[-0.88109279 -0.0489271   0.02975661  0.22558034]]
accuracy_score overall when tested on the training set: 0.7792207792207793
accuracy_score overall when tested on the testing set : 0.4687939934303144
